# CSCI 3155 : Assignment 8

__Name__:

Ian McGregor

In [327]:
// TEST HELPER: Do not EDIT this cell please.
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

passed: (points: Int)Unit


## Problem 1 : Type Checking (100 points)

Write a function `typecheck` that typechecks a Lettuce program.

$$\texttt{typecheck} : Program \rightarrow Type$$ 

This function should take as input a lettuce _Program_, and return what _Type_ it is. If it doesn't typecheck it should return a special value _TypeError_. Use the Lettuce typing notes on moodle for details on how typechecking lettuce works.

The tests for this assigment are hidden, so you will need to write your own tests! Note that you are encouraged to collaborate and share your test ideas on Piazza. To get full points, your implementation needs to match the specification from the Lettuce typing notes on moodle.

In [328]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType( ty_a : Type, ty_r : Type) extends Type
case object TypeError extends Type

sealed trait Expr
case class  Const(n : Int) extends Expr
case class  Ident(s : String) extends Expr
case object True extends Expr
case object False extends Expr
case class  Plus( e1 : Expr, e2 : Expr) extends Expr
case class  Minus( e1 : Expr, e2 : Expr) extends Expr
case class  Mult( e1 : Expr, e2 : Expr) extends Expr
case class  Neg( e1 : Expr) extends Expr
case class  And( e1 : Expr, e2 : Expr) extends Expr
case class  Or( e1 : Expr, e2 : Expr) extends Expr
case class  Geq( e1 : Expr, e2 : Expr) extends Expr
case class  Eq( e1 : Expr, e2 : Expr) extends Expr
case class  ITE( ec : Expr, et : Expr, ef : Expr) extends Expr
case class  Let( name : String, ty_d : Type, ed : Expr, ei : Expr) extends Expr
case class  FunDef( arg : String, ty_a : Type, eb : Expr) extends Expr
case class  FunCall( ef : Expr, ea : Expr) extends Expr
case class  LetRec( f : String, ty_f : Type, x : String, ty_x : Type, eb : Expr, ei : Expr) extends Expr

sealed trait Program
case class Main(p : Expr) extends Program

defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined object TypeError
defined trait Expr
defined class Const
defined class Ident
defined object True
defined object False
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined class And
defined class Or
defined class Geq
defined class Eq
defined class ITE
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined trait Program
defined class Main


In [329]:
type Environment = 
    Map[String, Type]

def TC (input : Expr, gamma : Environment) : Type = {
    input match {
        case Const(n) => n match {
            case n : Int => NumType
            }
        case Ident(s) => if(gamma.contains(s)) gamma(s) else TypeError
        case True => BoolType
        case False => BoolType
        case Plus(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match {
            case (NumType, NumType) => NumType
            case (_,_) => TypeError
        }
        case Minus(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match {
            case (NumType, NumType) => NumType
            case (_,_) => TypeError
        }
        case Mult(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match {
            case (NumType, NumType) => NumType
            case (_,_) => TypeError
        }
        case Neg(e1) => TC(e1,gamma) match {
            case TypeError => TypeError
            case NumType => NumType
            case BoolType => BoolType
            case FunType(a,b) => FunType(a,b)
        }
        case And(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match { 
            case (BoolType,BoolType) => BoolType
            case (_,_) => TypeError
        }
        case Or(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match { 
            case (BoolType,BoolType) => BoolType
            case (_,_) => TypeError
        }
        case Geq(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match {
            case (NumType, NumType) => BoolType
            case (_,_) => TypeError
        }
        case Eq(e1,e2) => (TC(e1,gamma) , TC(e2, gamma)) match {
            case (NumType, NumType) => BoolType
            case (BoolType, BoolType) => BoolType
            case (_,_) => TypeError
        }
        case ITE(ec, et , ef) => TC(ec, gamma) match {
            case TypeError => TypeError
            case NumType => TypeError
            case FunType(_,_) => TypeError
            case BoolType => (TC(et,gamma)) match {
                case TypeError => TypeError
                case et => if( et == TC(ef,gamma)){
                    et
                }else{TypeError}
            }
        }
        case Let(name, ty_d, ed, ei) => TC(ed,gamma) match {
            case TypeError => TypeError
            case ty_d => {
                val nugamma = gamma + (name -> TC(ed,gamma))
                TC(ei, nugamma)
            }
        }
        case FunDef(arg, ty_a, eb) => {
            val nugamma = gamma + (arg -> ty_a)
            TC(eb, nugamma) match {
                case TypeError => TypeError
                case tr => {
                    FunType(nugamma(arg), tr)
                }
            }
        }
        case FunCall(ef, ea) => TC(ef, gamma) match {
            case TypeError => TypeError
            case NumType => TypeError
            case BoolType => TypeError
            case FunType(ta, tr) => TC(ea, gamma) match {
                case ta => {
                    tr
                }
            }   
        }
        case LetRec(f_name, ty_f, arg_name, ty_arg, eb, ei) => {
            val nugamma = gamma + (arg_name -> ty_arg) + (f_name -> ty_f)
            TC(ei, nugamma) match {
                case TypeError => TypeError
                case ti => TC(eb,nugamma)
            }
        }
    }
}
def typecheck (program : Program) : Type = {
    val emptyenv : Environment = Map()
    program match {
        case Main(p) => TC(p, emptyenv)
    }
}

defined type alias Environment
TC: (input: Expr, gamma: Environment)Type
typecheck: (program: Program)Type


In [330]:
val p1 : Program = Main( Const(10) )
assert( typecheck(p1) == NumType, "test 1")
passed(20)
println("Hidden Tests : up to 80 points remaining")


*** Tests Passed (20 points) ***
Hidden Tests : up to 80 points remaining


null

In [331]:
println("Hidden test")

Hidden test


In [332]:
println("Hidden test")

Hidden test


In [333]:
println("Hidden test")

Hidden test


In [334]:
println("Hidden test")

Hidden test


In [335]:
println("Hidden test")

Hidden test


In [336]:
println("Hidden test")

Hidden test


In [337]:
println("Hidden test")

Hidden test


In [338]:
println("Hidden test")

Hidden test


In [339]:
println("Hidden test")

Hidden test


In [340]:
println("Hidden test")

Hidden test


In [341]:
println("Hidden test")

Hidden test


In [342]:
println("Hidden test")

Hidden test


In [343]:
println("Hidden test")

Hidden test


In [344]:
println("Hidden test")

Hidden test


In [345]:
println("Hidden test")

Hidden test


In [346]:
println("Hidden test")

Hidden test


In [347]:
println("Hidden test")

Hidden test


In [348]:
println("Hidden test")

Hidden test


In [349]:
println("Hidden test")

Hidden test


In [350]:
println("Hidden test")

Hidden test


In [351]:
println("Hidden test")

Hidden test


In [352]:
println("Hidden test")

Hidden test


In [353]:
println("Hidden test")

Hidden test


In [354]:
println("Hidden test")

Hidden test


In [355]:
println("Hidden test")

Hidden test


In [356]:
println("Hidden test")

Hidden test


In [357]:
println("Hidden test")

Hidden test


In [358]:
println("Hidden test")

Hidden test


In [359]:
println("Hidden test")

Hidden test


In [360]:
println("Hidden test")

Hidden test


In [361]:
println("Hidden test")

Hidden test


In [362]:
println("Hidden test")

Hidden test


In [363]:
println("Hidden test")

Hidden test
